## Импорты и загрузка данных

In [3]:
import numpy as np
import pandas as pd

import os

In [49]:
path = os.getcwd()

fish_df = pd.read_csv(path + r"\data\db1\ref\fish.csv", error_bad_lines=False, sep=';')
prod_designate_df = pd.read_csv(path + r"\data\db1\ref\prod_designate.csv", error_bad_lines=False, sep=';')
prod_type_df = pd.read_csv(path + r"\data\db1\ref\prod_type.csv", error_bad_lines=False, sep=';')
regime_df = pd.read_csv(path + r"\data\db1\ref\regime.csv", error_bad_lines=False, sep=';')
region_df = pd.read_csv(path + r"\data\db1\ref\region.csv", error_bad_lines=False, sep=';')


catch_df = pd.read_csv(path + r'\data\db1\catch.csv', error_bad_lines=False, sep=',')

# Подмена данных + нормализация
def spoofing(id):
    return fish_df[fish_df['id_fish'] == id]['fish'].values[0]
catch_df['fish'] = catch_df['id_fish'].apply(spoofing)
catch_df['catch_volume'] = catch_df['catch_volume']*1000

product_df = pd.read_csv(path + r'\data\db1\product.csv', error_bad_lines=False, sep=',')

ext1 = pd.read_csv(path + r"\data\db2\Ext.csv", error_bad_lines=False, sep=',')
ext2 = pd.read_csv(path + r"\data\db2\Ext2.csv", error_bad_lines=False, sep=',')

In [51]:
# Сводная таблица по всем поступлениям согласно IdFish и выбранного промежутка времени.
def calculatingSummOfFish(fish_id, date_start, date_end):
    # смотрим по всем выловам этой рыбы
    data = catch_df[catch_df['id_fish'] == fish_id]
    # уникальные номера кораблей ловящих только эту рыбу
    ships = list(set(data['id_ves'].to_list()))
    
    # создаю датафрейм с владельцами и выловом по судам    
    total_df = pd.DataFrame()
    
    total_ships = []
    total_owners = []
    total_catches = []
    
    # добавляем в общую кашу дату и документируем владельцев
    for ship in ships:
        ship_data = data[data['id_ves'] == ship]
        owners = list(set(ship_data['id_own'].to_list()))
        
        for owner in owners:
            ship_owner_data = ship_data[ship_data['id_own'] == owner]
            
            try:
                ship_owner_end_data = ship_owner_data[ship_owner_data['date'] < date_end]
                ship_owner_start_end_data = ship_owner_end_data[ship_owner_end_data['date'] > date_start]
                total_catch = ship_owner_start_end_data['catch_volume'].sum()
            except e:
                print(e)
            
            if total_catch > 0:
                total_catches.append(total_catch)
                total_ships.append(ship)
                total_owners.append(owner)
            
    total_df['total_catches'] = total_catches
    total_df['ship'] = total_ships
    total_df['owner'] = total_owners
    
    return total_df

### Поиск аномалий

#### Осуществим с помощью вычленения ID рыбы и даты вылова (для простоты возьмем весь период, но можно брать любой промежуток)

In [44]:
fish_id = input('Введите ID рыбы\n')

default_data = (catch_df['date'][0], catch_df['date'][len(catch_df) - 1])
start, end = default_data

#### Возьмем уникальный лист владельцев судов

In [62]:
catcher_owners = catch_df['id_own'].unique()
traider_owners = ext1['id_own'].unique()

market_owners = []
for owner in traider_owners:
    if owner in catcher_owners:
        market_owners.append(owner)  

In [61]:
print(len(catcher_owners), len(traider_owners), len(market_owners))

350 159 121


#### Как видим, db2 не полностью покрывает всех владельцев, следовательно, будем использовать для поиска только тех, которые задействованы в обоих БД

In [67]:
def calculatingSummOfFishTransfering(id, start, end):
    # Смотрю по каждому ID акты передачи    
    vsd_fish_id = ext2[ext2['fish'] == 'минтай']['id_vsd']
    # Для каждого акта определяю владельца     
    for vsd in vsd_fish_id:
        owners = ext1[ext1['id_vsd'] == vsd]['id_own']

In [68]:
calculatingSummOfFishTransfering('минтай', start, end)

3113575    3846064
Name: id_own, dtype: int64
3113576    999889
Name: id_own, dtype: int64
3113578    439595
Name: id_own, dtype: int64
3113620    6022174
Name: id_own, dtype: int64
3113621    6022174
Name: id_own, dtype: int64
3113622    6022174
Name: id_own, dtype: int64
3113624    6022174
Name: id_own, dtype: int64
3113636    2369769
Name: id_own, dtype: int64
3113637    2369769
Name: id_own, dtype: int64
3113638    2369769
Name: id_own, dtype: int64
3113639    2369769
Name: id_own, dtype: int64
3113647    3095952
Name: id_own, dtype: int64
3113685    6022174
Name: id_own, dtype: int64
3113801    6022174
Name: id_own, dtype: int64
3113338    9290322
Name: id_own, dtype: int64
3112929    5502023
3113339    7478978
Name: id_own, dtype: int64
3112927    5502023
3113340    7478978
Name: id_own, dtype: int64
3112926    5502023
3113341    7478978
Name: id_own, dtype: int64
3112868    5502023
3113342    6022174
Name: id_own, dtype: int64
3112865    5502023
3113343    6022174
Name: id_own, 

3041903    5502023
3090885    7115834
Name: id_own, dtype: int64
3040788    5502023
3068517    2369769
Name: id_own, dtype: int64
3040434    5502023
3068526    2369769
Name: id_own, dtype: int64
3040428    5502023
3068527    2369769
Name: id_own, dtype: int64
3040431    5502023
3068531    3095952
Name: id_own, dtype: int64
3039734    5502023
3068550    6462295
Name: id_own, dtype: int64
3038410    5502023
3068563    7636185
Name: id_own, dtype: int64
3022853    5502023
3042976    2476787
Name: id_own, dtype: int64
3022321    5502023
3042979    3107158
Name: id_own, dtype: int64
3022320    5502023
3042980    2476787
Name: id_own, dtype: int64
3020487    5502023
3042985    9290322
Name: id_own, dtype: int64
3019480    5502023
3090891    7115834
Name: id_own, dtype: int64
3019481    5502023
3105938    7115834
Name: id_own, dtype: int64
3019231    5502023
3042994    7478978
Name: id_own, dtype: int64
3019225    5502023
3042995    7478978
Name: id_own, dtype: int64
3019209    5502023
304299

2981319    5502023
2993289    4787153
Name: id_own, dtype: int64
2981317    5502023
2993301     439595
Name: id_own, dtype: int64
2981541    5502023
2993306    6545965
Name: id_own, dtype: int64
2981184    5502023
2993307    6022174
Name: id_own, dtype: int64
2981179    5502023
2993308    6022174
Name: id_own, dtype: int64
2981174    5502023
2993311    7636185
Name: id_own, dtype: int64
2980959    5502023
2993312    6948246
Name: id_own, dtype: int64
2981176    5502023
2993318    9290322
Name: id_own, dtype: int64
2981185    5502023
2993327    4787153
Name: id_own, dtype: int64
2981175    5502023
2993328    4787153
Name: id_own, dtype: int64
2981192    5502023
2993330     439595
Name: id_own, dtype: int64
2981183    5502023
2993331     439595
Name: id_own, dtype: int64
2980824    5502023
2993371    6022174
Name: id_own, dtype: int64
2980823    5502023
2993372    6022174
Name: id_own, dtype: int64
2980822    5502023
2993373    6022174
Name: id_own, dtype: int64
2980603    5502023
299337

2883230    5502023
2915516    7478978
Name: id_own, dtype: int64
2883219    5502023
2915517    7478978
Name: id_own, dtype: int64
2883204    5502023
2915518    7478978
Name: id_own, dtype: int64
2883205    5502023
2915519    7478978
Name: id_own, dtype: int64
2883019    5502023
2915520    6022174
Name: id_own, dtype: int64
2883018    5502023
2915521    6022174
Name: id_own, dtype: int64
2883017    5502023
2915522    6022174
Name: id_own, dtype: int64
2883015    5502023
2915523    6022174
Name: id_own, dtype: int64
2883202    5502023
2915524     999889
Name: id_own, dtype: int64
2883232    5502023
2915527    7115834
Name: id_own, dtype: int64
2883225    5502023
2947617    7115834
Name: id_own, dtype: int64
2883216    5502023
2993661    7115834
Name: id_own, dtype: int64
2883217    5502023
2982002    7115834
Name: id_own, dtype: int64
2883213    5502023
2947620    7115834
Name: id_own, dtype: int64
2883193    5502023
2915530    7115834
Name: id_own, dtype: int64
2883186    5502023
291553

2820808    5502023
2852696    2369769
Name: id_own, dtype: int64
2820584    5502023
2852720    2369769
Name: id_own, dtype: int64
2820581    5502023
2852725    7629399
Name: id_own, dtype: int64
2820582    5502023
2852726    7629399
Name: id_own, dtype: int64
2820150    5502023
2852736    6377225
Name: id_own, dtype: int64
2811663    5502023
2822391    2476787
Name: id_own, dtype: int64
2810136    5502023
2822399    9290322
Name: id_own, dtype: int64
2809714    5502023
2822400    7478978
Name: id_own, dtype: int64
2809619    5502023
2822401    7478978
Name: id_own, dtype: int64
2809713    5502023
2822402    7478978
Name: id_own, dtype: int64
2809712    5502023
2822403    7478978
Name: id_own, dtype: int64
2809615    5502023
2822404     999889
Name: id_own, dtype: int64
2809706    5502023
2822406    7115834
Name: id_own, dtype: int64
2809708    5502023
2822408    7115834
Name: id_own, dtype: int64
2809715    5502023
2822412    4787153
Name: id_own, dtype: int64
2809612    5502023
282242

...

1539497    5502023
1556097     439595
Name: id_own, dtype: int64
1499977    5502023
1539948     439595
Name: id_own, dtype: int64
1499978    5502023
1539947     439595
Name: id_own, dtype: int64
1539516    5502023
1556095     439595
Name: id_own, dtype: int64
1539517    5502023
1556096     439595
Name: id_own, dtype: int64
1500723    5502023
1539925    3846064
Name: id_own, dtype: int64
1536440    5502023
1556254    4882414
Name: id_own, dtype: int64
1537568    5502023
1556253    4882414
Name: id_own, dtype: int64
1532538    5502023
1556325    4882414
Name: id_own, dtype: int64
1532537    5502023
1556326    4882414
Name: id_own, dtype: int64
1535707    5502023
1556285    4882414
Name: id_own, dtype: int64
1534212    5502023
1556298    4882414
Name: id_own, dtype: int64
900154    5502023
Name: id_own, dtype: int64
923696    5502023
Name: id_own, dtype: int64
923697    5502023
Name: id_own, dtype: int64
923698    5502023
Name: id_own, dtype: int64
923758    5502023
Name: id_own, dtype: i

1486137    5502023
Name: id_own, dtype: int64
535033    5502023
Name: id_own, dtype: int64
514366    5502023
Name: id_own, dtype: int64
509647    5502023
Name: id_own, dtype: int64
1277727    5502023
Name: id_own, dtype: int64
2580036    5502023
Name: id_own, dtype: int64
874749    5502023
Name: id_own, dtype: int64
874217    5502023
Name: id_own, dtype: int64
2121159    5502023
Name: id_own, dtype: int64
2125399    5502023
Name: id_own, dtype: int64
2814070    5502023
Name: id_own, dtype: int64
2817041    5502023
Name: id_own, dtype: int64
1659616    5502023
Name: id_own, dtype: int64
1661531    5502023
Name: id_own, dtype: int64
806542    5502023
Name: id_own, dtype: int64
1632308    5502023
Name: id_own, dtype: int64
1834557    5502023
Name: id_own, dtype: int64
2805340    5502023
Name: id_own, dtype: int64
2804712    5502023
Name: id_own, dtype: int64
2603594    5502023
Name: id_own, dtype: int64
2604255    5502023
Name: id_own, dtype: int64
729513    5502023
Name: id_own, dtype: i

2747638    5502023
Name: id_own, dtype: int64
2752135    5502023
Name: id_own, dtype: int64
2751641    5502023
Name: id_own, dtype: int64
2746963    5502023
Name: id_own, dtype: int64
2751905    5502023
Name: id_own, dtype: int64
1664545    5502023
Name: id_own, dtype: int64
1659878    5502023
Name: id_own, dtype: int64
1662034    5502023
Name: id_own, dtype: int64
1664384    5502023
Name: id_own, dtype: int64
1660067    5502023
Name: id_own, dtype: int64
1665010    5502023
Name: id_own, dtype: int64
1660393    5502023
Name: id_own, dtype: int64
2265718    5502023
Name: id_own, dtype: int64
2268729    5502023
Name: id_own, dtype: int64
2409368    5502023
Name: id_own, dtype: int64
2408174    5502023
Name: id_own, dtype: int64
2409723    5502023
Name: id_own, dtype: int64
2412467    5502023
Name: id_own, dtype: int64
2409800    5502023
Name: id_own, dtype: int64
979719    5502023
Name: id_own, dtype: int64
974821    5502023
Name: id_own, dtype: int64
975077    5502023
Name: id_own, dtyp

2395705    5502023
Name: id_own, dtype: int64
2393596    5502023
Name: id_own, dtype: int64
2390077    5502023
Name: id_own, dtype: int64
2396383    5502023
Name: id_own, dtype: int64
2398104    5502023
Name: id_own, dtype: int64
2396143    5502023
Name: id_own, dtype: int64
2647878    5502023
Name: id_own, dtype: int64
2637305    5502023
Name: id_own, dtype: int64
2629278    5502023
Name: id_own, dtype: int64
2645203    5502023
Name: id_own, dtype: int64
2640195    5502023
Name: id_own, dtype: int64
2398791    5502023
Name: id_own, dtype: int64
2398916    5502023
Name: id_own, dtype: int64
2397778    5502023
Name: id_own, dtype: int64
2396093    5502023
Name: id_own, dtype: int64
1980179    5502023
Name: id_own, dtype: int64
1978210    5502023
Name: id_own, dtype: int64
1973132    5502023
Name: id_own, dtype: int64
1972261    5502023
Name: id_own, dtype: int64
1980882    5502023
Name: id_own, dtype: int64
1980184    5502023
Name: id_own, dtype: int64
1978521    5502023
Name: id_own, d

KeyboardInterrupt: 

In [45]:
def getAllOwnersIncomings(id, start, end):
    concstructive_table = calculatingSummOfFish(id, start, end)
    

'2022-01-01'

In [64]:
ext1

,id_fishery,id_own,date_fishery,numPart,id_Plat,id_vsd,Name_Plat,Product_period,Region_Plat
0,2208303,6493928,2022-05-24 00:00:00,\N,5022636,8745666,\N,\N,\N
1,5538439,3846064,2022-05-24 00:00:00,\N,839830,3846313,\N,\N,\N
2,8688535,3107158,2022-05-24 00:00:00,\N,9310751,5215676,\N,\N,\N
3,8688535,3107158,2022-05-24 00:00:00,\N,9310751,4008716,\N,\N,\N
4,9102536,6493928,2022-05-24 00:00:00,\N,2900344,8522784,\N,\N,\N
...,...,...,...,...,...,...,...,...,...
3260793,2140098,7372847,2020-12-30 00:00:00,\N,4176694,2113660,\N,\N,\N
3260794,3458281,7115834,2020-12-29 00:00:00,\N,906811,6111511,\N,\N,\N
3260795,3458281,7115834,2020-12-29 00:00:00,\N,906811,3138810,\N,\N,\N
3260796,3458281,7115834,2020-12-29 00:00:00,\N,906811,2081767,\N,\N,\N
